___________________________

<p align="center" style="font-size: 30px">Modificações sem o atributo "Perímetro"</p>

___________________

In [ ]:
import pandas as pd

# Nomes das colunas
column_names = [
    "Área",
    "Perímetro",
    "Compacidade",
    "Comprimento_grão",
    "Largura_grão",
    "Coeficiente_assimetria",
    "Comprimento_sulco",
    "Classe"
]

# Leitura do arquivo CSV (separado por tabulação ou espaço)
df = pd.read_csv('seeds_dataset.csv', header=None, delim_whitespace=True, names=column_names)

df

C:\Users\gFire\AppData\Local\Temp\ipykernel_4736\1304445059.py:16: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv('seeds_dataset.csv', header=None, delim_whitespace=True, names=column_names)


,Área,Perímetro,Compacidade,Comprimento_grão,Largura_grão,Coeficiente_assimetria,Comprimento_sulco,Classe
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,1
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,1
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,1
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,1
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,1
...,...,...,...,...,...,...,...,...
205,12.19,13.20,0.8783,5.137,2.981,3.631,4.870,3
206,11.23,12.88,0.8511,5.140,2.795,4.325,5.003,3
207,13.20,13.66,0.8883,5.236,3.232,8.315,5.056,3
208,11.84,13.21,0.8521,5.175,2.836,3.598,5.044,3


# Exclusão de Atributos

In [21]:
df_changed = df.copy()

# Exclui a coluna Perímetro
df_changed.drop(columns=["Perímetro"], inplace=True)

# Salve o DataFrame modificado em um novo arquivo CSV
df_changed.to_csv('seeds_dataset_modified.csv', index=False)

# Exibe o DataFrame modificado
df_changed

,Área,Compacidade,Comprimento_grão,Largura_grão,Coeficiente_assimetria,Comprimento_sulco,Classe
0,15.26,0.8710,5.763,3.312,2.221,5.220,1
1,14.88,0.8811,5.554,3.333,1.018,4.956,1
2,14.29,0.9050,5.291,3.337,2.699,4.825,1
3,13.84,0.8955,5.324,3.379,2.259,4.805,1
4,16.14,0.9034,5.658,3.562,1.355,5.175,1
...,...,...,...,...,...,...,...
205,12.19,0.8783,5.137,2.981,3.631,4.870,3
206,11.23,0.8511,5.140,2.795,4.325,5.003,3
207,13.20,0.8883,5.236,3.232,8.315,5.056,3
208,11.84,0.8521,5.175,2.836,3.598,5.044,3


# Adição de Ruído 



In [ ]:
import pandas as pd
import numpy as np

# Nomes das colunas
column_names = [
    "Área",
    "Compacidade",
    "Comprimento_grão",
    "Largura_grão",
    "Coeficiente_assimetria",
    "Comprimento_sulco",
    "Classe"
]

# Criar cópia para adicionar ruído
df_noisy1 = df_changed.copy()

# Adicionar ruído (exceto à coluna 'class')
noise_std = 0.1
for col in df_noisy1.columns[:-1]:  # Ignora a última coluna ('class')
    df_noisy1[col] = df_noisy1[col] + np.random.normal(0, noise_std, size=len(df_noisy1))

# Salvar
df_noisy1.to_csv('datasets/seeds_noisy1.csv', index=False, header=False)

# Carregar com nomes das colunas atualizadas (sem 'perimeter')
noisy_df1 = pd.read_csv('datasets/seeds_noisy1.csv', header=None, names=column_names)

# Exibir
noisy_df1

,Área,Compacidade,Comprimento_grão,Largura_grão,Coeficiente_assimetria,Comprimento_sulco,Classe
0,15.234397,0.818298,5.823909,3.338266,2.147357,5.304079,1
1,14.856177,0.845195,5.550194,3.588291,0.965166,5.009900,1
2,14.331663,0.761266,5.261106,3.331742,2.680296,5.031640,1
3,13.835371,0.821771,5.361886,3.216729,2.145196,4.963958,1
4,16.162812,0.916950,5.618893,3.556664,1.384353,5.195389,1
...,...,...,...,...,...,...,...
205,12.089426,0.925657,5.381790,3.038217,3.638465,4.856924,3
206,11.191175,0.794297,5.206084,2.994115,4.577158,4.983947,3
207,13.246805,0.783358,5.230685,3.237037,8.417763,5.070177,3
208,11.801715,0.863186,5.144290,2.835336,3.718105,5.020523,3


# Normalização


In [ ]:
import pandas as pd

# Nomes das colunas
column_names = [
    "Área",
    "Compacidade",
    "Comprimento_grão",
    "Largura_grão",
    "Coeficiente_assimetria",
    "Comprimento_sulco",
    "Classe"
]

# Criar cópia para normalizar
normalized_df = df_changed.copy()

# Normalização Min-Max (6 primeiras colunas)
for i in range(6):  # Índices 0 a 6
    min_val = normalized_df.iloc[:, i].min()  # Acessa coluna por POSIÇÃO (iloc)
    max_val = normalized_df.iloc[:, i].max()
    normalized_df.iloc[:, i] = (normalized_df.iloc[:, i] - min_val) / (max_val - min_val)

# Salvar
normalized_df.to_csv('datasets/seeds_normalized1.csv', index=False, header=False)

# Carregar o resultado com nomes de colunas (opcional)
df_normalized = pd.read_csv('datasets/seeds_normalized1.csv', header=None, names=column_names)

# Exibir
df_normalized


,Área,Compacidade,Comprimento_grão,Largura_grão,Coeficiente_assimetria,Comprimento_sulco,Classe
0,0.440982,0.570780,0.486486,0.486101,0.189302,0.345150,1
1,0.405099,0.662432,0.368806,0.501069,0.032883,0.215165,1
2,0.349386,0.879310,0.220721,0.503920,0.251453,0.150665,1
3,0.306893,0.793103,0.239302,0.533856,0.194243,0.140817,1
4,0.524079,0.864791,0.427365,0.664291,0.076701,0.322994,1
...,...,...,...,...,...,...,...
205,0.151086,0.637024,0.134009,0.250178,0.372635,0.172821,3
206,0.060434,0.390200,0.135698,0.117605,0.462872,0.238306,3
207,0.246459,0.727768,0.189752,0.429081,0.981667,0.264402,3
208,0.118036,0.399274,0.155405,0.146828,0.368344,0.258493,3


# Padronização

In [ ]:
import pandas as pd

# Nomes das colunas
column_names = [
    "Área",
    "Compacidade",
    "Comprimento_grão",
    "Largura_grão",
    "Coeficiente_assimetria",
    "Comprimento_sulco",
    "Classe"
]

# Criar cópia para padronização
standardized_df = df_changed.copy()

# Padronização (Z-Score) nas 6 primeiras colunas
for i in range(6): 
    mean = standardized_df.iloc[:, i].mean()
    std = standardized_df.iloc[:, i].std()
    standardized_df.iloc[:, i] = (standardized_df.iloc[:, i] - mean) / std

# Salvar (sem cabeçalho)
standardized_df.to_csv('datasets/seeds_standardized1.csv', index=False, header=False)

# Carregar e exibir com nomes das colunas
df_standardized = pd.read_csv('datasets/seeds_standardized1.csv', header=None, names=column_names)
df_standardized

,Área,Compacidade,Comprimento_grão,Largura_grão,Coeficiente_assimetria,Comprimento_sulco,Classe
0,0.141759,0.000060,0.303493,0.141364,-0.983801,-0.382663,1
1,0.011161,0.427494,-0.168223,0.196962,-1.783904,-0.919816,1
2,-0.191609,1.438945,-0.761817,0.207552,-0.665888,-1.186357,1
3,-0.346264,1.036904,-0.687336,0.318747,-0.958528,-1.227051,1
4,0.444196,1.371233,0.066507,0.803240,-1.559768,-0.474223,1
...,...,...,...,...,...,...,...
205,-0.913333,0.308997,-1.109397,-0.734959,-0.046025,-1.094797,3
206,-1.243264,-0.842110,-1.102626,-1.227395,0.415547,-0.824186,3
207,-0.566218,0.732199,-0.885953,-0.070436,3.069254,-0.716349,3
208,-1.033620,-0.799790,-1.023631,-1.118847,-0.067973,-0.740765,3


# Discretização

In [ ]:
import pandas as pd

# Nomes das colunas
column_names = [
    "Área",
    "Compacidade",
    "Comprimento_grão",
    "Largura_grão",
    "Coeficiente_assimetria",
    "Comprimento_sulco",
    "Classe"
]

# Criar cópia para descretização
descretized_df = df_changed.copy()

# Lista com os nomes das colunas (exceto a classe)
features = descretized_df.columns[:-1]  # ignora a última coluna ("classe")

# Discretizar cada coluna em 3 categorias
for col in features:
    descretized_df[col] = pd.cut(descretized_df[col], bins=3, labels=["baixo", "médio", "alto"])

    
# Salvar
descretized_df.to_csv('datasets/seeds_discretized1.csv', index=False, header=False)

# Carregar o resultado com nomes de iunas (opcional)
df_discretized = pd.read_csv('datasets/seeds_discretized1.csv', header=None, names=column_names)

# Exibir
df_discretized


,Área,Compacidade,Comprimento_grão,Largura_grão,Coeficiente_assimetria,Comprimento_sulco,Classe
0,médio,médio,médio,médio,baixo,médio,1
1,médio,médio,médio,médio,baixo,baixo,1
2,médio,alto,baixo,médio,baixo,baixo,1
3,baixo,alto,baixo,médio,baixo,baixo,1
4,médio,alto,médio,médio,baixo,baixo,1
...,...,...,...,...,...,...,...
205,baixo,médio,baixo,baixo,médio,baixo,3
206,baixo,médio,baixo,baixo,médio,baixo,3
207,baixo,alto,baixo,médio,alto,baixo,3
208,baixo,médio,baixo,baixo,médio,baixo,3


# Redução de Dimensionalidade

In [ ]:
import pandas as pd
from sklearn.decomposition import PCA

# Nomes das colunas (sem 'perimeter')
column_names = [
    "Área",
    "Compacidade",
    "Comprimento_grão",
    "Largura_grão",
    "Coeficiente_assimetria",
    "Comprimento_sulco",
    "Classe"
]

# Criar cópia para PCA
df_PCA = df_changed.copy()

X = df_PCA.iloc[:, :6].values
y = df_PCA.iloc[:, 6].values

# Aplicar PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

# Criar novo DataFrame
PCA_df = pd.DataFrame(X_pca, columns=['PC1', 'PC2'])
PCA_df['class'] = y

# Salvar
PCA_df.to_csv('datasets/seeds_pca1.csv', index=False, header=False)

# Carregar o resultado com nomes de colunas (opcional)
df_pca_view = pd.read_csv('datasets/seeds_pca1.csv', header=None, names=['PC1', 'PC2', 'Classe'])

# Exibir o DataFrame PCA
df_pca_view

,PC1,PC2,Classe
0,0.611929,-1.416760,1
1,0.358993,-2.683969,1
2,-0.508409,-1.123153,1
3,-0.870089,-1.616895,1
4,1.594515,-2.161596,1
...,...,...,...
205,-2.722980,-0.483447,3
206,-3.751541,0.085576,3
207,-2.365891,4.293189,3
208,-3.043006,-0.546589,3


# Introdução de Valores Ausentes 

In [ ]:
import pandas as pd
import numpy as np

# Nomes das colunas (sem 'perimeter')
column_names = [
    "Área",
    "Compacidade",
    "Comprimento_grão",
    "Largura_grão",
    "Coeficiente_assimetria",
    "Comprimento_sulco",
    "Classe"
]

# Criar cópia para adicionar dados faltantes
df_missing_data = df_changed.copy()

# Adicionar NaN em 5% dos valores (exceto classe)
mask = np.random.random(df_missing_data.shape) < 0.05
mask[:, 6] = False  # Preservar coluna de classe
df_missing_data[mask] = np.nan

# Salvar
df_missing_data.to_csv('datasets/seeds_missing1.csv', index=False, header=False, na_rep='NaN')

# Carregar o resultado com nomes de colunas (opcional)
df_missing_view = pd.read_csv('datasets/seeds_missing1.csv', header=None, names=column_names)

# Exibir o DataFrame com dados faltantes
df_missing_view

,Área,Compacidade,Comprimento_grão,Largura_grão,Coeficiente_assimetria,Comprimento_sulco,Classe
0,15.26,0.8710,5.763,3.312,2.221,5.220,1
1,14.88,0.8811,5.554,3.333,1.018,4.956,1
2,14.29,0.9050,5.291,3.337,2.699,4.825,1
3,13.84,0.8955,5.324,3.379,2.259,4.805,1
4,16.14,0.9034,5.658,3.562,NaN,5.175,1
...,...,...,...,...,...,...,...
205,12.19,0.8783,5.137,2.981,3.631,4.870,3
206,11.23,0.8511,5.140,NaN,4.325,5.003,3
207,13.20,0.8883,5.236,NaN,8.315,5.056,3
208,11.84,0.8521,5.175,2.836,3.598,5.044,3


# Desbalanceamento de Classes

In [ ]:
import pandas as pd

# Nomes das colunas (sem 'perimeter')
column_names = [
    "Área",
    "Compacidade",
    "Comprimento_grão",
    "Largura_grão",
    "Coeficiente_assimetria",
    "Comprimento_sulco",
    "Classe"
]

# Criar cópia para desbalanceamento
df_unbalanced = df_changed.copy()

# Reduzir classe 1 para 30 amostras (original tem 70)
class_1 = df_unbalanced[df_unbalanced['Classe'] == 1].sample(n=30, random_state=42)
df_balanced = pd.concat([class_1, df_unbalanced[df_unbalanced['Classe'] != 1]])

# Salvar
df_balanced.to_csv('datasets/seeds_unbalanced1.csv', index=False, header=False)

# Carregar o resultado com nomes de colunas (opcional)
df_unbalanced_view = pd.read_csv('datasets/seeds_unbalanced1.csv', header=None, names=column_names)

# Exibir o DataFrame desbalanceado
df_unbalanced_view


,Área,Compacidade,Comprimento_grão,Largura_grão,Coeficiente_assimetria,Comprimento_sulco,Classe
0,15.88,0.8988,5.618,3.507,0.7651,5.091,1
1,15.26,0.8710,5.763,3.312,2.2210,5.220,1
2,14.86,0.8676,5.678,3.258,2.1290,5.351,1
3,16.14,0.9034,5.658,3.562,1.3550,5.175,1
4,14.52,0.8557,5.741,3.113,1.4810,5.487,1
...,...,...,...,...,...,...,...
165,12.19,0.8783,5.137,2.981,3.6310,4.870,3
166,11.23,0.8511,5.140,2.795,4.3250,5.003,3
167,13.20,0.8883,5.236,3.232,8.3150,5.056,3
168,11.84,0.8521,5.175,2.836,3.5980,5.044,3


# Transformação Não-Linear

In [ ]:
import pandas as pd
import numpy as np

# Nomes das colunas (sem 'perimeter')
column_names = [
    "Área",
    "Compacidade",
    "Comprimento_grão",
    "Largura_grão",
    "Coeficiente_assimetria",
    "Comprimento_sulco",
    "Classe"
]

# Criar cópia para transformação logarítmica
df_transformed = df_changed.copy()

# Aplicar transformação logarítmica a todas as colunas exceto a de classe
for col in df_transformed.columns[:-1]:  # ignora a coluna 'classe'
    df_transformed[col] = np.log(df_transformed[col] + 1)

# Salvar
df_transformed.to_csv('datasets/seeds_log1.csv', index=False, header=False)

# Carregar o resultado com nomes de colunas (opcional)
df_log_view = pd.read_csv('datasets/seeds_log1.csv', header=None, names=column_names)

# Exibir o DataFrame transformado
df_log_view

,Área,Compacidade,Comprimento_grão,Largura_grão,Coeficiente_assimetria,Comprimento_sulco,Classe
0,2.788708,0.626473,1.911467,1.461402,1.169692,1.827770,1
1,2.765060,0.631857,1.880076,1.466260,0.702107,1.784399,1
2,2.727199,0.644482,1.839120,1.467183,1.308063,1.762159,1
3,2.697326,0.639483,1.844352,1.476820,1.181420,1.758720,1
4,2.841415,0.643642,1.895819,1.517761,0.856541,1.820509,1
...,...,...,...,...,...,...,...
205,2.579459,0.630367,1.814336,1.381533,1.532773,1.769855,3
206,2.503892,0.615780,1.814825,1.333684,1.672413,1.792259,3
207,2.653242,0.635677,1.830339,1.442675,2.231626,1.801050,3
208,2.552565,0.616320,1.820509,1.344430,1.525621,1.799066,3
